## Import Autophagy Package

In [ ]:
# Specific imports for cellular_raza
# The package is named cr_autophagy
# We want to reload the package when some of the behind-the scenes python functions change
# This is what the importlib statements are for
import importlib
import cr_autophagy as cra
importlib.reload(cra)

# Imports of general-purpose python libraries
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import pyvista as pv
import multiprocessing as mp
from pathlib import Path
import os

output_path = cra.get_last_output_path()
simulation_settings = cra.get_simulation_settings(output_path)
iter_0_particles = cra.get_particles_at_iter(output_path, 0)
for col in iter_0_particles.columns:
    print(col)
max_iter = max(cra.get_all_iterations(output_path))

## Read results from json Files
The results of the simulation are saved in json files.
Due to the parallelized nature of the simulation, not all results are in one big json file but rather in multiple batches. We therefore need to combine these batches to obtain a complete set for a given iteration.

We want to inspect which entries our generated dataset has. Therefore, we normalize the dict, transforming it into a dataframe.
Afterwards, we display all columns.

## Analysis
1. Determine how the Cargo size changes over the course of the simulation
2. Determine if ATG11Receptor actually clusters

In [ ]:
df = cra.get_particles_at_iter(output_path, max_iter)

cargo_at_end = df[df["element.cell.interaction.species"]=="Cargo"]["element.cell.mechanics.mechanics.pos"]
cargo_at_end = np.array([np.array(elem) for elem in cargo_at_end])
non_cargo_at_end = df[df["element.cell.interaction.species"]!="Cargo"]["element.cell.mechanics.mechanics.pos"]
non_cargo_at_end = np.array([np.array(elem) for elem in non_cargo_at_end])

In [ ]:
# Save all scatter snapshots
cra.save_all_scatter_snapshots(output_path, threads=-1)

# Also create a movie with ffmpeg
bashcmd = f"ffmpeg -y -r 30 -f image2 -pattern_type glob -i '{output_path}/scatterplots/*.png' -c:v h264 -pix_fmt yuv420p -strict -2 {output_path}/scatter_movie.mp4"
os.system(bashcmd)

## Plot Result
We visualize the results in 3D.
Therefore we use `pyvista` which internally uses `vtk` as a backend.
Since all of our particles are represented as 3D-spheres, we also display them as such.

We can save single snapshots or even use all processes of our device to save snapshots for every iteration.
The 2nd approach will take up all resources by default. If you want to limit this, have a look at the [Pool object of the multiprocessing](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.pool.Pool) module.

In [ ]:
# Save all snapshots
cra.save_all_snapshots(output_path, threads=-1)

# Also create a movie with ffmpeg
bashcmd = f"ffmpeg -y -r 30 -f image2 -pattern_type glob -i '{output_path}/snapshots/*.png' -c:v h264 -pix_fmt yuv420p -strict -2 {output_path}/snapshot_movie.mp4"
os.system(bashcmd)